Create summary of the requested runs.

In [38]:
import os
import json
import yaml
import numpy as np
from pathlib import Path
from tensorboard.backend.event_processing.event_file_loader import EventFileLoader
from collections import OrderedDict

In [28]:
from_run = 1703
to_run = 6000

In [29]:
def get_run_dict(paths_dict: dict) -> dict:
    result = {}
    #dir_contents: list = []
    for key in paths_dict:
        if key != "working_dir":
            dir_contents = os.listdir(paths_dict[key])
    
            #numbers = []
            #rest_of_file_name: str = ""
            for entry in dir_contents:
                number = 0
                if "_" in entry:
                    number = int(entry.split("_")[0])
                    #rest_of_file_name = str.join("_", entry.split("_")[1:])
                    
                else:
                    number = int(entry.split(".")[0])
                    #rest_of_file_name = str.join(".", entry.split(".")[1:])
                    
                if number not in result:
                    result[number] = {}
                    
                result[number][key] = paths_dict[key] / f"{entry}"
    #if not numbers:
    #    return []
    
            #for num in numbers:
                
    
    # Remove double entries and sort.
    #result = dict.fromkeys(numbers)
    #result.sort()

    #for key in result:
    #    result[key] = {}
    #    result[key]["summary_path"] = paths_dict["summaries_dir"] / f"{key}_summary.json"
    #    result[key]["result_path"] = paths_dict["summaries_dir"] / f"{key}_summary.json"

    return result

In [35]:
def get_mean_reward_std(path: Path) -> tuple[float, float]:
    """Get the mean reward over the last 5 cumulative rewards entries in the tfevents file."""
    cumulative_rewards = []

    # Get the tfevents file associated with the current run.
    path_to_result_folder = path / "RollerAgent/"
    try:
        path_to_result = sorted(Path(path_to_result_folder).glob("events.out.tfevents.*"))[0]
    except:
        return 0,0

    # Using tensorflow to access the tfevents data.
    # datarecord = EventFileLoader(str(path_to_result)).Load()
    for event in EventFileLoader(str(path_to_result)).Load():
        # event = event_pb2.Event.FromString(batch.numpy())
        for value in event.summary.value:
            if value.tag == "Environment/Cumulative Reward":
                cumulative_rewards.append(value.tensor.float_val[0])

    # Return mean of the last 5 recorded cummulative rewards.
    rewards_of_interest = cumulative_rewards[-100:]
    return np.mean(rewards_of_interest), np.round(np.std(rewards_of_interest), 10)

In [31]:
# Enter paths for plot creation and get absolute paths.
paths = {
        "working_dir": "C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env",
        "results_dir": "results/",
        "results_archive_dir": "results_archive/",
        "stats_dir": "stats/",
        "summaries_dir": "summaries/",
        "configs_dir": "configs/",
        "unity_configs_dir": "unity_configs/"
    }

paths["working_dir"] = Path(paths["working_dir"]).absolute()

for key in paths:
    if key != "working_dir":
        paths[key] = paths["working_dir"] / paths[key]
        

paths

{'working_dir': WindowsPath('C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env'),
 'results_dir': WindowsPath('C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env/results'),
 'results_archive_dir': WindowsPath('C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env/results_archive'),
 'stats_dir': WindowsPath('C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env/stats'),
 'summaries_dir': WindowsPath('C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env/summaries'),
 'configs_dir': WindowsPath('C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env/configs'),
 'unity_configs_dir': WindowsPath('C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env/unity_configs')}

In [32]:
run_dict = get_run_dict(paths)

selected_ids = [x for x in run_dict.keys() if from_run <= x <= to_run]
selected_ids




[1703,
 1706,
 1751,
 1758,
 1770,
 1793,
 1808,
 1842,
 1891,
 1895,
 1928,
 1949,
 1968,
 2032,
 2045,
 2270,
 2303,
 2323,
 2341,
 2352,
 2439,
 2503,
 2518,
 2560,
 2573,
 2699,
 2744,
 2958,
 2979,
 2996,
 3005,
 3035,
 3036,
 3061,
 3087,
 3091,
 3108,
 3134,
 3192,
 3243,
 3272,
 3295,
 3378,
 3548,
 3572,
 3579,
 3580,
 3632,
 3657,
 3717,
 3734,
 3751,
 3765,
 3788,
 3818,
 3876,
 3923,
 4015,
 4110,
 4121,
 4192,
 4204,
 4209,
 4244,
 4289,
 4342,
 4376,
 4392,
 4479,
 4499,
 4571,
 4602,
 4654,
 4776,
 4888,
 5005,
 5009,
 5027,
 5043,
 5102,
 5136,
 5168,
 5204,
 5208,
 5227,
 5246,
 5266,
 5315,
 5316,
 5330,
 5370,
 5385,
 5395,
 5413,
 5426,
 5430,
 5529,
 5617,
 5629,
 5635,
 5662,
 5679,
 5754,
 5809,
 5866,
 5941,
 5956]

In [39]:
selected_ids_dict = {}
for run_id in selected_ids:
    selected_ids_dict[run_id] = {}

    if "stats_dir" in run_dict[run_id]:
        with open(run_dict[run_id]["stats_dir"]) as json_file:
        #results["stats"] = json.load(json_file)
            selected_ids_dict[run_id]["stats"] = json.load(json_file)
    
    if "unity_configs_dir" in run_dict[run_id]:
        with open(run_dict[run_id]["unity_configs_dir"]) as json_file:
        #results["stats"] = json.load(json_file)
            selected_ids_dict[run_id]["unity_config"] = json.load(json_file)

    if "configs_dir" in run_dict[run_id]:
        with open(run_dict[run_id]["configs_dir"]) as yaml_file:
        #results["stats"] = json.load(json_file)
            selected_ids_dict[run_id]["training_config"] = yaml.safe_load(yaml_file)
    
    if "results_dir" in run_dict[run_id]:
        selected_ids_dict[run_id]["mean_reward"], selected_ids_dict[run_id]["std_reward"] = get_mean_reward_std(run_dict[run_id]["results_dir"])

#selected_ids_dict[4015]

In [40]:
sorted_dict = OrderedDict(
            sorted(
                selected_ids_dict.items(), key=lambda v: v[1]["mean_reward"], reverse=True
            )
        )
sorted_dict

OrderedDict([(1703,
              {'unity_config': {'sensorCount': 1,
                'useDecoy': False,
                'createWall': True,
                'doorWidth': 4.0,
                'randomWallPosition': True,
                'randomDoorPosition': True,
                'targetAlwaysInOtherRoomFromAgent': False,
                'targetFixedPosition': False,
                'maxStep': 1000,
                'stepPenalty': -0.0005,
                'runId': 1703,
                'statsExportPath': 'C:\\Users\\max.muehlefeldt\\Documents\\GitHub\\unity-machine-learning\\python\\basic_rl_env\\stats\\1703_stats.json'},
               'training_config': {'behaviors': {'RollerAgent': {'hyperparameters': {'batch_size': 248,
                   'beta': '1e-2',
                   'buffer_size': 51200,
                   'epsilon': 0.2,
                   'lambd': 0.95,
                   'learning_rate': '1e-5',
                   'learning_rate_schedule': 'linear',
                   'num_e